In [3]:
# %load symmetric_encryption.py
#!/usr/bin/env/python3

import urllib.request
from Crypto.Cipher import AES
import binascii
import base64
import random
import os
import zlib

################################################################################
# CS 284 Padding Utility Functions
################################################################################

# s is a bytearray to pad, k is blocklength
# you won't need to change the block length
def cmsc284pad(s,k=16):
  if k > 255:
    print("pkcs7pad: padding block length must be less than 256")
    return bytearray()
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s.extend([i])
  return s

# s is bytes to pad, k is blocklength
# you won't need to change the block length
def cmsc284padbytes(s,k=16):
  if k > 255:
    raise Exception("pkcs7pad: padding block length must be less than 256")
  n = k - (len(s) % k)
  if n == 0:
    n = k
  for i in range(1,n+1):
    s += chr(i).encode("utf-8")
  return s

# s is bytes to unpad, k is blocklength
# you won't need to change the block length
def cmsc284unpad(s,k=16):
  if not cmsc284checkpadding(s,k):
    print("cmsc284unpad: invalid padding")
    return b''
  n = s[len(s)-1]
  return s[:len(s)-n]

# checks padding on s and returns a boolean
# you won't need to change the block length
def cmsc284checkpadding(s,k=16):
  if(len(s) == 0):
    #print("Invalid padding: String zero length"%k) 
    return False
  if(len(s)%k != 0): 
    #print("Invalid padding: String is not multiple of %d bytes"%k) 
    return False
  n = s[len(s)-1]
  if n > k or n == 0:
    return False
  else: 
    for i in range(n):
      if s[len(s)-1-i] != (n-i):
        return False
  return True

################################################################################
# Function for querying the server
################################################################################

PPS2SERVER = "http://cryptoclass.cs.uchicago.edu/"
def make_query(task, cnetid, query):
  DEBUG = False
  if DEBUG:
    print("making a query")
    print("Task:", task)
    print("CNET ID:", cnetid)
    print("Query:", query)
  if (type(query) is bytearray) or (type(query) is bytes):
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query)) + "/"
  else:
    url = PPS2SERVER + urllib.parse.quote_plus(task) + "/" + urllib.parse.quote_plus(cnetid) + "/" + urllib.parse.quote_plus(base64.urlsafe_b64encode(query.encode('utf-8'))) + "/"
  if DEBUG:
    print("Querying:", url)

  with urllib.request.urlopen(url) as response:
    raw_answer = response.read()
    answer = base64.urlsafe_b64decode(raw_answer)
    if DEBUG:
      print("Answer:", answer)
    return answer
  return None


################################################################################
# Problem 1 SOLUTION
################################################################################

def problem1(cnetid):
  return b''


################################################################################
# Problem 2 SOLUTION
################################################################################

def problem2(cnetid):
  return b''


################################################################################
# Problem 3 SOLUTION
################################################################################

def problem3(cnetid):
  return b''


################################################################################
# Problem 4 SOLUTION
################################################################################

def problem4(cnetid):
  return b''


################################################################################
# Problem 5 SOLUTION
################################################################################

def problem5(cnetid):
  return b''

################################################################################
# Problem 6 SOLUTION
################################################################################

def problem6(cnetid):
  return b''

In [4]:
from copy import deepcopy

In [17]:
def get_aes_dec_byte(cnetid, ctext, idx):
  """modify ctext in place and
  return dec_byte = AES^{-1}(k, c[i])[idx]"""
  dec_byte = None
  for byte in range(1, 256):
    ctext[idx] = byte
    if idx > 0:
      # perturb the previous byte to avoid false positive
      ctext[idx - 1] = 0
    resp = make_query('fiveb', cnetid, ctext)
    if resp == b'true':
      dec_byte = ctext[idx] ^ 1
      break
  return dec_byte

In [12]:
def set_ctext(ctext, recovered, start, end):
  """modify in place"""
  for i in range(end - start):
    offset = start + i
    byte = (i + 2) % 256 # wrap around
    ctext[offset] = recovered[offset] ^ byte

In [13]:
def problem5(cnetid):
  ctext = bytearray(make_query('fivea', cnetid, ''))
  len_ctext = len(ctext)
  len_flag = len_ctext - 16
  aes_dec_arr = bytearray(len_flag)
  flag_arr = bytearray(len_flag)
  # first block can't be used on its own
  for block_idx in range(len_ctext, 16, -16):
    ctext_mod = ctext[:block_idx] # new copy
    offset = len(ctext_mod) - 16
    for idx in range(offset - 1, offset - 17, -1):
      dec_byte = get_aes_dec_byte(cnetid, ctext_mod, idx)
      aes_dec_arr[idx] = dec_byte
      flag_arr[idx] = dec_byte ^ ctext[idx]
      print(idx, bytes([flag_arr[idx]]))
      set_ctext(ctext_mod, aes_dec_arr, idx, offset)
  if not cmsc284checkpadding(flag_arr):
    raise
  else:
    return cmsc284unpad(flag_arr)

In [19]:
problem5('ruolinzheng')

31 b'\x05'
30 b'\x04'
29 b'\x03'
28 b'\x02'
27 b'\x01'
26 b's'
25 b'o'
24 b'h'
23 b't'
22 b' '
21 b'o'
20 b'T'
19 b' '
18 b'.'
17 b'y'
16 b'a'
15 b'D'
14 b' '
13 b's'
12 b'n'
11 b'a'
10 b'r'
9 b'e'
8 b't'
7 b'e'
6 b'V'
5 b' '
4 b'y'
3 b'p'
2 b'p'
1 b'a'
0 b'H'


bytearray(b'Happy Veterans Day. To thos')

In [18]:
problem5('ffalzon')

31 b'\x05'
30 b'\x04'
29 b'\x03'
28 b'\x02'
27 b'\x01'
26 b'e'
25 b'f'
24 b'e'
23 b'f'
22 b'v'
21 b'o'
20 b'c'
19 b' '
18 b','
17 b'g'
16 b'n'
15 b'i'
14 b'h'
13 b't'
12 b'y'
11 b'r'
10 b'e'
9 b'v'
8 b'e'
7 b' '
6 b'e'
5 b't'
4 b'i'
3 b'p'
2 b's'
1 b'e'
0 b'D'


bytearray(b'Despite everything, covfefe')

In [8]:
ctext = bytearray(make_query('fivea', 'ffalzon', ''))
len_flag = len(ctext) - 16
aes_dec_arr = bytearray(len_flag)
flag_arr = bytearray(len_flag)

block_end_idx = 48
ctext_mod = ctext[:block_end_idx]
flag_arr_idx = block_end_idx - 16
for idx in range(flag_arr_idx - 1, flag_arr_idx - 17, -1):
  dec_byte = get_aes_dec_byte('ffalzon', ctext_mod, idx)
  aes_dec_arr[idx] = dec_byte
  flag_arr[idx] = dec_byte ^ ctext[idx]
  print(idx, bytes([flag_arr[idx]]))
  set_ctext(ctext_mod, aes_dec_arr, idx, flag_arr_idx)

ctext_mod = ctext[:32]
len_msg = len(ctext_mod) - 16
for idx in range(len_msg - 1, len_msg - 17, -1):
  dec_byte = get_aes_dec_byte('ffalzon', ctext_mod, idx)
  aes_dec_arr[idx] = dec_byte
  flag_arr[idx] = dec_byte ^ ctext[idx]
  print(idx, bytes([flag_arr[idx]]))
  set_ctext(ctext_mod, aes_dec_arr, idx, len_msg)
flag_arr

31 b'\x01'


TypeError: an integer is required